In [10]:
import re
import string
import os
import gc
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
from datetime import datetime as dt

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder
from category_encoders.one_hot import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
from sqlalchemy.engine.base import Engine
from yaml import safe_load

from api.database.database import engine
from api.config.config import PG_USER, PG_PASS, PG_HOST, PG_PORT, PG_DATABASE


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 100)

In [11]:
def select(query: str, conn: Engine=engine) -> pd.DataFrame:
    return pd.read_sql(query, conn)
    
users_query = "SELECT * FROM public.user_data"
posts_query = "SELECT * FROM public.post_text_df"
feeds_query = "SELECT * FROM public.feed_data WHERE action = 'view' LIMIT 5000000"

users_df = select(users_query)
posts_df = select(posts_query)
feeds_df = select(feeds_query)

In [12]:
users_df.shape, posts_df.shape, feeds_df.shape

((163205, 8), (7023, 3), (5000000, 5))

In [13]:
posts_df.head(2)

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufacturing sector will continue to face serious chall...,business
1,2,Aids and climate top Davos agenda\n\nClimate change and the fight against Aids are leading the l...,business


In [14]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')


punctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


def preprocessing(text):
    tokens_list = word_tokenize(text.lower())
    punctuation_free = [token for token in tokens_list if token not in punctuation]
    stop_words_free = [token for token in punctuation_free if token not in stop_words]
    digits_free = [token for token in stop_words_free if not re.search(r'\d', token)]
    lemmatized = " ".join([lemmatizer.lemmatize(word) for word in digits_free]) 
    return lemmatized


vectorizer = TfidfVectorizer(preprocessor=preprocessing)
transformed_output = vectorizer.fit_transform(posts_df['text']).toarray()
tfidf_df = pd.DataFrame(
    transformed_output,
    index=posts_df.post_id,
    columns=vectorizer.get_feature_names_out())

posts_df['TotalTfIdf'] = tfidf_df.sum(axis=1).reset_index()[0]
posts_df['MaxTfIdf'] = tfidf_df.max(axis=1).reset_index()[0]
posts_df['MeanTfIdf'] = tfidf_df.mean(axis=1).reset_index()[0]


centered = tfidf_df - tfidf_df.mean(axis=0)
pca = PCA(n_components=30)
pca_decomp = pca.fit_transform(centered)


kmeans = KMeans(n_clusters=20, random_state=0).fit(pca_decomp)
posts_df['TextCluster'] = kmeans.labels_
dists_columns = ['DistanceTo1thCluster',
                 'DistanceTo2thCluster',
                 'DistanceTo3thCluster',
                 'DistanceTo4thCluster',
                 'DistanceTo5thCluster',
                 'DistanceTo6thCluster',
                 'DistanceTo7thCluster',
                 'DistanceTo8thCluster',
                 'DistanceTo9thCluster',
                 'DistanceTo10thCluster',
                 'DistanceTo11thCluster',
                 'DistanceTo12thCluster',
                 'DistanceTo13thCluster',
                 'DistanceTo14thCluster',
                 'DistanceTo15thCluster',
                 'DistanceTo16thCluster',
                 'DistanceTo17thCluster',
                 'DistanceTo18thCluster',
                 'DistanceTo19thCluster',
                 'DistanceTo20thCluster']
kmeans_df = pd.DataFrame(
    data=kmeans.transform(pca_decomp),
    columns=dists_columns
)
kmeans_df.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/radionnazmiev/nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     /home/radionnazmiev/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/radionnazmiev/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,DistanceTo7thCluster,DistanceTo8thCluster,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster,DistanceTo16thCluster,DistanceTo17thCluster,DistanceTo18thCluster,DistanceTo19thCluster,DistanceTo20thCluster
0,0.536210,0.453265,0.514293,0.498317,0.635344,0.147062,0.557242,0.457470,0.468691,0.401456,0.549420,0.494289,0.559193,0.445537,0.528308,0.465897,0.548300,0.639463,0.453182,0.582998
1,0.433877,0.334106,0.401162,0.381961,0.570874,0.352258,0.461465,0.328709,0.335174,0.254984,0.391468,0.374893,0.465714,0.227272,0.441335,0.344476,0.415282,0.366821,0.316760,0.495487
2,0.462180,0.384519,0.430278,0.424902,0.587703,0.265839,0.496152,0.364003,0.386108,0.229368,0.520180,0.408047,0.512229,0.372413,0.473460,0.377241,0.455532,0.608120,0.374774,0.516313
3,0.452198,0.386751,0.430632,0.406762,0.580223,0.382999,0.472190,0.364827,0.376813,0.239100,0.498241,0.408623,0.506734,0.355798,0.480493,0.363230,0.459092,0.603930,0.367499,0.471592
4,0.333833,0.226672,0.293962,0.263888,0.520903,0.338223,0.364091,0.179787,0.215616,0.102403,0.413133,0.264623,0.398379,0.214971,0.362458,0.195010,0.365515,0.503222,0.219658,0.406047


In [15]:
posts_df = pd.concat((posts_df,kmeans_df),axis=1)
posts_df.head(2)

,post_id,text,topic,TotalTfIdf,MaxTfIdf,MeanTfIdf,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,DistanceTo7thCluster,DistanceTo8thCluster,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster,DistanceTo16thCluster,DistanceTo17thCluster,DistanceTo18thCluster,DistanceTo19thCluster,DistanceTo20thCluster
0,1,UK economy facing major risks\n\nThe UK manufacturing sector will continue to face serious chall...,business,9.015487,0.489283,0.000208,5,0.536210,0.453265,0.514293,0.498317,0.635344,0.147062,0.557242,0.457470,0.468691,0.401456,0.549420,0.494289,0.559193,0.445537,0.528308,0.465897,0.548300,0.639463,0.453182,0.582998
1,2,Aids and climate top Davos agenda\n\nClimate change and the fight against Aids are leading the l...,business,12.082760,0.313700,0.000279,13,0.433877,0.334106,0.401162,0.381961,0.570874,0.352258,0.461465,0.328709,0.335174,0.254984,0.391468,0.374893,0.465714,0.227272,0.441335,0.344476,0.415282,0.366821,0.316760,0.495487


In [16]:
posts_df.to_sql(
   "posts_info_by_radion_nazmiev",
    con=engine,
    schema="public",
    if_exists='replace'
)

23

In [17]:
del users_query
del posts_query
del feeds_query
del nltk
del punctuation
del stop_words
del lemmatizer
del vectorizer
del transformed_output
del tfidf_df
del centered
del pca
del pca_decomp
del kmeans
del dists_columns
del kmeans_df
del PCA
del KMeans

gc.collect()

319

In [18]:
df = pd.merge(
    feeds_df,
    posts_df,
    on='post_id',
    how='inner'
)
df = pd.merge(
    users_df,
    df,
    on='user_id',
    how='inner'
)

df.shape

(5000000, 38)

In [19]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['hour'] = df['timestamp'].dt.hour
df['month'] = df['timestamp'].dt.month

del feeds_df
del posts_df
del users_df

df = df.set_index(['user_id', 'post_id'])

df.head(2)

gender  age     country  city  exp_group   os source  \
user_id post_id                                                         
6288    4800          0   38  Kazakhstan  Esik          2  iOS    ads   
        3345          0   38  Kazakhstan  Esik          2  iOS    ads   

                          timestamp action  target  \
user_id post_id                                      
6288    4800    2021-12-07 16:12:31   view       0   
        3345    2021-12-01 12:50:00   view       0   

                                                                                                                text  \
user_id post_id                                                                                                        
6288    4800     Hillary Swank is an unattractive piece of work in this unattractive piece of work of a film. Pat...   
        3345     America :FACT Kamala Is DEFUND POLICE  Don’t be fooled by her slick Willie Talk! #PoliceLivesMat...   

                 topic  TotalTfIdf  MaxTfIdf  MeanTfIdf  TextCluster  \
user_id post_id                                                        
6288    4800     movie    6.834736  0.299565   0.000158           11   
        3345     covid    3.250926  0.386018   0.000075           15   

                 DistanceTo1thCluster  DistanceTo2thCluster  \
user_id post_id                                               
6288    4800                 0.371225              0.287472   
        3345                 0.278161              0.240087   

                 DistanceTo3thCluster  DistanceTo4thCluster  \
user_id post_id                                               
6288    4800                 0.275530              0.306575   
        3345                 0.290842              0.179015   

                 DistanceTo5thCluster  DistanceTo6thCluster  \
user_id post_id                                               
6288    4800                 0.558034              0.408391   
        3345                 0.543505              0.376018   

                 DistanceTo7thCluster  DistanceTo8thCluster  \
user_id post_id                                               
6288    4800                 0.402759              0.166127   
        3345                 0.312335              0.173045   

                 DistanceTo9thCluster  DistanceTo10thCluster  \
user_id post_id                                                
6288    4800                 0.262185               0.272899   
        3345                 0.236570               0.214021   

                 DistanceTo11thCluster  DistanceTo12thCluster  \
user_id post_id                                                 
6288    4800                  0.458697               0.083002   
        3345                  0.442479               0.262532   

                 DistanceTo13thCluster  DistanceTo14thCluster  \
user_id post_id                                                 
6288    4800                  0.352831               0.295778   
        3345                  0.407298               0.252060   

                 DistanceTo15thCluster  DistanceTo16thCluster  \
user_id post_id                                                 
6288    4800                  0.397676               0.254181   
        3345                  0.374327               0.049105   

                 DistanceTo17thCluster  DistanceTo18thCluster  \
user_id post_id                                                 
6288    4800                  0.412422               0.550310   
        3345                  0.404184               0.535272   

                 DistanceTo19thCluster  DistanceTo20thCluster  hour  month  
user_id post_id                                                             
6288    4800                  0.305265               0.452710    16     12  
        3345                  0.283820               0.365045    12     12

In [21]:
max(df.timestamp), min(df.timestamp)

(Timestamp('2021-12-29 23:51:06'), Timestamp('2021-10-01 06:05:25'))

In [22]:
df_train = df[df.timestamp < '2021-12-15']
df_test = df[df.timestamp >= '2021-12-15']

df_train = df_train.drop(['timestamp','action', 'text'], axis=1)
df_test = df_test.drop(['timestamp','action', 'text'], axis=1)

X_train = df_train.drop('target', axis=1)
X_test = df_test.drop('target', axis=1)

y_train = df_train['target']
y_test = df_test['target']
del df
del df_train
del df_test

y_train.shape, y_test.shape

((4169234,), (830766,))

In [23]:
object_cols = [
    'topic', 'TextCluster', 'gender', 'country',
    'city', 'exp_group', 'hour', 'month',
    'os', 'source'
]

cols_for_ohe = [x for x in object_cols if X_train[x].nunique() < 5]
cols_for_mte = [x for x in object_cols if X_train[x].nunique() >= 5]


cols_for_ohe_idx = [list(X_train.columns).index(col) for col in cols_for_ohe]
cols_for_mte_idx = [list(X_train.columns).index(col) for col in cols_for_mte]

t = [
    ('OneHotEncoder', OneHotEncoder(), cols_for_ohe_idx),
    ('MeanTargetEncoder', TargetEncoder(), cols_for_mte_idx)
]

col_transform = ColumnTransformer(transformers=t)


pipe_dt = Pipeline([("column_transformer",
                     col_transform),

                    ("decision_tree",
                     DecisionTreeClassifier())])

pipe_dt.fit(X_train, y_train)

/media/radionnazmiev/Data/DS/recommendation_system_project/venv/lib/python3.8/site-packages/category_encoders/target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/media/radionnazmiev/Data/DS/recommendation_system_project/venv/lib/python3.8/site-packages/category_encoders/target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."
/media/radionnazmiev/Data/DS/recommendation_system_project/venv/lib/python3.8/site-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
/media/radionnaz

Pipeline(steps=[('column_transformer',
                 ColumnTransformer(transformers=[('OneHotEncoder',
                                                  OneHotEncoder(),
                                                  [0, 33, 5, 6]),
                                                 ('MeanTargetEncoder',
                                                  TargetEncoder(),
                                                  [7, 11, 2, 3, 4, 32])])),
                ('decision_tree', DecisionTreeClassifier())])

In [24]:
print(f"Качество на трейне: {roc_auc_score(y_train, pipe_dt.predict_proba(X_train)[:, 1])}")
print(f"Качество на тесте: {roc_auc_score(y_test, pipe_dt.predict_proba(X_test)[:, 1])}")

Качество на трейне: 0.9206731670305771
Качество на тесте: 0.5337221026718026


In [25]:
catboost = CatBoostClassifier(iterations=100,
                              learning_rate=1,
                              depth=2,
                              random_seed=100)

catboost.fit(X_train, y_train, object_cols, logging_level='Verbose')

/media/radionnazmiev/Data/DS/recommendation_system_project/venv/lib/python3.8/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


0:	learn: 0.3556279	total: 619ms	remaining: 1m 1s
1:	learn: 0.3485846	total: 1.08s	remaining: 53s
2:	learn: 0.3474588	total: 1.55s	remaining: 50s
3:	learn: 0.3466656	total: 1.95s	remaining: 46.9s
4:	learn: 0.3464465	total: 2.4s	remaining: 45.7s
5:	learn: 0.3462855	total: 2.79s	remaining: 43.7s
6:	learn: 0.3462204	total: 3.14s	remaining: 41.7s
7:	learn: 0.3459938	total: 3.51s	remaining: 40.3s
8:	learn: 0.3452651	total: 3.88s	remaining: 39.2s
9:	learn: 0.3452360	total: 4.24s	remaining: 38.1s
10:	learn: 0.3449363	total: 4.65s	remaining: 37.6s
11:	learn: 0.3448905	total: 5.02s	remaining: 36.8s
12:	learn: 0.3445261	total: 5.39s	remaining: 36.1s
13:	learn: 0.3444972	total: 5.75s	remaining: 35.3s
14:	learn: 0.3442369	total: 6.14s	remaining: 34.8s
15:	learn: 0.3441972	total: 6.52s	remaining: 34.2s
16:	learn: 0.3441740	total: 6.91s	remaining: 33.7s
17:	learn: 0.3438409	total: 7.3s	remaining: 33.2s
18:	learn: 0.3438314	total: 7.66s	remaining: 32.7s
19:	learn: 0.3437613	total: 8.07s	remaining: 32

In [26]:
print(f"Качество на трейне: {roc_auc_score(y_train, catboost.predict_proba(X_train)[:, 1])}")
print(f"Качество на тесте: {roc_auc_score(y_test, catboost.predict_proba(X_test)[:, 1])}")

Качество на трейне: 0.667011765571919


/media/radionnazmiev/Data/DS/recommendation_system_project/venv/lib/python3.8/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


Качество на тесте: 0.6489843651058338


In [27]:
catboost.save_model('catboost_model')